# Financial News Summarization Agent - Step by Step

This notebook walks through the process of using the Financial News Summarization Agent to retrieve, summarize, and evaluate financial news articles related to a stock portfolio.

We'll follow these steps:
1. Setup and imports
2. Initialize the vector database connection
3. Initialize summarization models (baseline and fine-tuned)
4. Retrieve articles for a portfolio
5. Generate summaries using both models
6. Evaluate the quality of summaries
7. Visualize and compare results

## 1. Setup and Imports

In [14]:
# Standard imports
import os
import sys
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# ROUGE metrics for evaluation
from rouge_score import rouge_scorer

# Add the parent directory to path to import our scripts
sys.path.append(os.path.join(os.path.dirname('__file__'), ".."))
from scripts.vector_db_manager import VectorDatabaseManager

# LangChain imports
from langchain.llms import OpenAI, HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

# Configure paths
DB_PATH = "../data/chroma_db"
COLLECTION_NAME = "apple_financial_articles"
MODEL_DIR = "../models"

# Create data directory if it doesn't exist
os.makedirs("../data", exist_ok=True)


## 2. Initialize Vector Database

First, we'll connect to our vector database to retrieve relevant financial news articles.

In [12]:
# Initialize the vector database connection
vector_db = VectorDatabaseManager(
    db_path=DB_PATH,
    collection_name=COLLECTION_NAME
)

# Check how many documents are in the collection
collection_count = vector_db.collection.count()
print(f"Collection '{COLLECTION_NAME}' contains {collection_count} documents")


Initializing ChromaDB with persistence directory: ../data/chroma_db
Loaded existing collection 'apple_financial_articles'
Collection 'apple_financial_articles' contains 1194 documents
Collection 'apple_financial_articles' contains 1194 documents


## 3. Initialize Summarization Models

Now, we'll set up both baseline and fine-tuned summarization models. We'll use LangChain to create a flexible pipeline that can work with different model backends.

In [16]:
# Setup ROUGE scorer for evaluation
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize baseline model using HuggingFace Hub
# Note: If you have an OpenAI API key, you can use that instead
try:
    # Check if environment variable exists
    huggingface_api_token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
    openai_api_key = os.environ.get("OPENAI_API_KEY")
    
    if openai_api_key:
        # Use OpenAI as baseline if API key is available
        baseline_llm = OpenAI(temperature=0.3, openai_api_key=openai_api_key)
        print("Using OpenAI as baseline model")
    else:
        # Use an open-source model from HuggingFace as fallback
        baseline_llm = HuggingFaceHub(
            repo_id="facebook/bart-large-cnn", 
            huggingfacehub_api_token=huggingface_api_token,
            model_kwargs={"temperature": 0.3, "max_length": 150}
        )
        print("Using HuggingFace BART model as baseline")
    
    # Create baseline prompt template
    baseline_prompt = PromptTemplate(
        input_variables=["text"],
        template="""Summarize the following financial news article in a concise, factual manner:

{text}

Summary:"""
    )
    
    # Create baseline summarization chain
    # baseline_chain = LLMChain(
    #     llm=baseline_llm,
    #     prompt=baseline_prompt
    # )
    baseline_chain = baseline_prompt | baseline_llm
except Exception as e:
    print(f"Error initializing model: {str(e)}")
    print("\nAlternative: You can install a local model like 't5-small' to proceed without API keys.")
    baseline_chain = None


Using HuggingFace BART model as baseline


In [17]:
# For fine-tuned model, we would normally load a model from disk
# For this demo, we'll use a different pre-trained model as a stand-in

try:
    # Check if we have a locally fine-tuned model
    finetuned_model_path = os.path.join(MODEL_DIR, "finetuned_summarizer")
    
    if os.path.exists(finetuned_model_path):
        # Load the local fine-tuned model
        finetuned_llm = HuggingFaceHub(
            repo_id=finetuned_model_path,
            huggingfacehub_api_token=huggingface_api_token,
            model_kwargs={"temperature": 0.3, "max_length": 150}
        )
        print(f"Using fine-tuned model from {finetuned_model_path}")
    else:
        # Use a different pre-trained model as our "fine-tuned" model for demonstration
        finetuned_llm = HuggingFaceHub(
            repo_id="sshleifer/distilbart-cnn-12-6",  # smaller, "fine-tuned" version
            huggingfacehub_api_token=huggingface_api_token,
            model_kwargs={"temperature": 0.3, "max_length": 150}
        )
        print("Using DistilBART CNN as simulated fine-tuned model")
    
    # Create finetuned prompt template (similar to baseline for this demo)
    finetuned_prompt = PromptTemplate(
        input_variables=["text"],
        template="""Summarize this financial news article focusing on key market insights, company performance, and investor implications:

{text}

Summary:"""
    )
    
    # Create finetuned summarization chain
    finetuned_chain = LLMChain(
        llm=finetuned_llm,
        prompt=finetuned_prompt
    )
    
except Exception as e:
    print(f"Error initializing fine-tuned model: {str(e)}")
    finetuned_chain = None


Using DistilBART CNN as simulated fine-tuned model


## 4. Retrieve Articles for Portfolio

Now, let's define our portfolio of stocks and retrieve relevant articles from the vector database.

In [18]:
def get_articles_for_portfolio(tickers, days_back=7, article_limit=5):
    """
    Retrieve articles related to the stock portfolio from the vector database.
    
    Args:
        tickers: List of stock tickers to search for
        days_back: How many days back to search
        article_limit: Max number of articles per ticker
        
    Returns:
        List of dictionaries containing article info
    """
    all_articles = []
    cutoff_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
    
    for ticker in tickers:
        # Create a query to find articles about this ticker
        query = f"{ticker} financial news stock market"
        print(f"Searching for articles about {ticker}...")
        
        # Search the vector database
        results = vector_db.query_database(
            query=query,
            n_results=article_limit,
            metadata_filter=None,  # Can filter by date here if needed
            include_summary=True
        )
        
        # Process and filter results
        documents = results.get("documents", [])
        metadatas = results.get("metadatas", [])
        distances = results.get("distances", [])
        
        # Handle nested result structure from ChromaDB
        if documents and isinstance(documents, list) and documents and isinstance(documents[0], list):
            documents = documents[0]  # Take first query results
            
        if metadatas and isinstance(metadatas, list) and metadatas and isinstance(metadatas[0], list):
            metadatas = metadatas[0]  # Take first query results
            
        if distances and isinstance(distances, list) and distances and isinstance(distances[0], list):
            distances = distances[0]  # Take first query results
            
        # Add articles to results
        if documents:
            for i, doc in enumerate(documents):
                metadata = metadatas[i]
                article_info = {
                    "ticker": ticker,
                    "document": doc,
                    "metadata": metadata,
                    "similarity": 1.0 - distances[i]
                }
                all_articles.append(article_info)
    
    # Sort by similarity score (descending)
    all_articles.sort(key=lambda x: x["similarity"], reverse=True)
    
    print(f"Retrieved {len(all_articles)} relevant articles in total")
    return all_articles

# Define our portfolio
portfolio = ["AAPL", "TSLA", "MSFT"]

# Retrieve articles
articles = get_articles_for_portfolio(
    tickers=portfolio,
    days_back=30,  # Look back 30 days to ensure we find some articles
    article_limit=3  # Limit to 3 articles per ticker for this demo
)


Searching for articles about AAPL...
Found 3 results. Showing top 3:

Result #1 (Similarity: 0.4040)
Title: LGT Group Foundation Cuts Holdings in Apple Inc. (NASDAQ:AAPL)
Source: Americanbankingnews
Date: 2025-05-12 07:12:58
Chunk: 14 of 14
Link: https://www.americanbankingnews.com/2025/05/12/lgt-group-foundation-cuts-holdings-in-apple-inc-nasdaqaapl.html
Creator: ABMN Staff
Summary: LGT Group Foundation lowered its stake in shares of Apple Inc. (NASDAQ:AAPL – Free Report) by 1.3% during the 4th quarter, according to the company in its most recent filing with the SEC.
Apple accounts for 7.0% of LGT Group Foundation’s holdings, making the stock its 2nd biggest holding.
LGT Group Foundation’s holdings in Apple were worth $453,877,000 as of its most recent filing with the SEC.
Menard Financial Group LLC raised its position in shares of Apple by 0.4% during the 3rd quarter.
Hanseatic Management Services Inc. lifted its holdings in shares of Apple by 1.3% during the 4th quarter.
Article ID:

Let's look at a few example articles that we retrieved:

In [19]:
if articles:
    # Display information about the first few articles
    for i, article in enumerate(articles[:3]):
        print(f"Article #{i+1} - {article['ticker']} (Similarity: {article['similarity']:.4f})")
        print(f"Title: {article['metadata'].get('title', 'N/A')}")
        print(f"Source: {article['metadata'].get('source', 'N/A')}")
        print(f"Date: {article['metadata'].get('pubDate', 'N/A')}")
        print("\nExcerpt:")
        print(article['document'][:300] + "...")
        print("-" * 80)
else:
    print("No articles found. Make sure your vector database contains financial news articles.")


Article #1 - AAPL (Similarity: 0.4040)
Title: LGT Group Foundation Cuts Holdings in Apple Inc. (NASDAQ:AAPL)
Source: Americanbankingnews
Date: 2025-05-12 07:12:58

Excerpt:
Recommended Stories

Want to see what other hedge funds are holding AAPL? Visit HoldingsChannel.com to get the latest 13F filings and insider trades for Apple Inc. (NASDAQ:AAPL – Free Report).

Receive News & Ratings for Apple Daily - Enter your email address below to receive a concise daily summary...
--------------------------------------------------------------------------------
Article #2 - AAPL (Similarity: 0.4032)
Title: DDFG Inc Decreases Stock Holdings in Apple Inc. (NASDAQ:AAPL)
Source: Defenseworld Net
Date: 2025-05-13 07:04:50

Excerpt:
Get Our Latest Stock Analysis on AAPL

Insider Buying and Selling...
--------------------------------------------------------------------------------
Article #3 - AAPL (Similarity: 0.3889)
Title: Scratch Capital LLC Acquires New Position in Apple Inc. (NASDAQ:AAPL)
Source:

## 5. Generate Summaries

Now, let's generate summaries for the retrieved articles using both our baseline and fine-tuned models.

In [28]:
baseline_llm.invoke(input='Hello How are you?')

AttributeError: 'InferenceClient' object has no attribute 'post'

In [31]:
from langchain.llms.huggingface_endpoint import HuggingFaceEndpoint

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="facebook/bart-large-cnn",
    huggingfacehub_api_token=huggingface_api_token,
    task="summarization"x,
    model_kwargs={"max_length": 150}
)

In [45]:
articles[0]['metadata']['summary']

'LGT Group Foundation lowered its stake in shares of Apple Inc. (NASDAQ:AAPL – Free Report) by 1.3% during the 4th quarter, according to the company in its most recent filing with the SEC.\nApple accounts for 7.0% of LGT Group Foundation’s holdings, making the stock its 2nd biggest holding.\nLGT Group Foundation’s holdings in Apple were worth $453,877,000 as of its most recent filing with the SEC.\nMenard Financial Group LLC raised its position in shares of Apple by 0.4% during the 3rd quarter.\nHanseatic Management Services Inc. lifted its holdings in shares of Apple by 1.3% during the 4th quarter.'

In [46]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    token=huggingface_api_token  # or set via environment variable
)


result = summarizer(articles[0]['metadata']['summary'], max_length=150, min_length=30, do_sample=False)
print(result[0]['summary_text'])

Device set to use mps:0


LGT Group Foundation lowered its stake in shares of Apple Inc. (NASDAQ:AAPL – Free Report) by 1.3% during the 4th quarter. Apple accounts for 7.0% of LGT Group Foundation’s holdings, making the stock its 2nd biggest holding. The firm owned $453,877,000 as of its most recent filing with the SEC.


In [29]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceEndpoint
import os

# Set your Hugging Face API token
huggingface_api_token = os.environ.get("HUGGINGFACEHUB_API_TOKEN")

# Create the LLM
llm = HuggingFaceEndpoint(
    repo_id="facebook/bart-large-cnn",
    huggingfacehub_api_token=huggingface_api_token,
    task="summarization",
    model_kwargs={"max_length": 150}
)

# Create a prompt template
prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize the following:\n\n{text}\n\nSummary:"
)

# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)



In [30]:
# Run a query
input_text = "Apple Inc. reported a significant increase in quarterly earnings, driven by strong iPhone sales and growth in its services segment."
result = chain.run({"text": input_text})
print(result)

AttributeError: 'InferenceClient' object has no attribute 'post'

In [21]:
def generate_summaries(articles):
    """
    Generate summaries for a list of articles using both baseline and fine-tuned models.
    
    Args:
        articles: List of articles to summarize
        
    Returns:
        List of articles with summaries added
    """
    for article in tqdm(articles, desc="Generating summaries"):
        text = article["document"]
        
        # Generate baseline summary
        if baseline_chain:
            try:
                article["baseline_summary"] = baseline_chain.invoke(text)
            except Exception as e:
                print(f"Error generating baseline summary: {str(e)}")
                article["baseline_summary"] = "Error generating summary."
        
        # Generate fine-tuned summary
        if finetuned_chain:
            try:
                article["finetuned_summary"] = finetuned_chain.invoke(text)
            except Exception as e:
                print(f"Error generating fine-tuned summary: {str(e)}")
                article["finetuned_summary"] = "Error generating summary."
        
    return articles

# Generate summaries for all articles
# Note: This may take some time depending on the number of articles and model access
if baseline_chain or finetuned_chain:
    articles_with_summaries = generate_summaries(articles[:3])  # Just summarize first 3 for demo
else:
    print("Cannot generate summaries: models are not available")


Generating summaries:   0%|          | 0/3 [00:00<?, ?it/s]

Error generating baseline summary: 'InferenceClient' object has no attribute 'post'
Error generating fine-tuned summary: 'InferenceClient' object has no attribute 'post'
Error generating baseline summary: 'InferenceClient' object has no attribute 'post'
Error generating fine-tuned summary: 'InferenceClient' object has no attribute 'post'
Error generating baseline summary: 'InferenceClient' object has no attribute 'post'
Error generating fine-tuned summary: 'InferenceClient' object has no attribute 'post'


Let's look at the generated summaries for one of the articles:

In [ ]:
if 'articles_with_summaries' in locals() and articles_with_summaries:
    article = articles_with_summaries[0]  # First article
    
    print(f"Article about {article['ticker']}")
    print(f"Title: {article['metadata'].get('title', 'N/A')}")
    print("\nORIGINAL EXCERPT:")
    print(article['document'][:500] + "..." if len(article['document']) > 500 else article['document'])
    
    print("\n" + "-" * 40)
    print("BASELINE SUMMARY:")
    print(article.get('baseline_summary', 'No baseline summary available'))
    
    print("\n" + "-" * 40)
    print("FINE-TUNED SUMMARY:")
    print(article.get('finetuned_summary', 'No fine-tuned summary available'))


## 6. Evaluate Summaries

Now, let's evaluate the quality of our summaries using ROUGE metrics. This will give us a quantitative measure of how well our models are performing.

In [ ]:
def evaluate_summaries(articles):
    """
    Evaluate the quality of generated summaries using Rouge metrics.
    
    Args:
        articles: List of articles with generated summaries
        
    Returns:
        Dictionary with evaluation metrics
    """
    # Initialize metrics
    metrics = {
        "baseline": {"rouge1": [], "rouge2": [], "rougeL": []},
        "finetuned": {"rouge1": [], "rouge2": [], "rougeL": []},
        "comparison": {"better": 0, "worse": 0, "same": 0}
    }
    
    for article in articles:
        # Use the article's summary (if available) as reference
        reference = article["metadata"].get("summary", "")
        
        if reference and "baseline_summary" in article and "finetuned_summary" in article:
            # Score baseline summary
            baseline_scores = scorer.score(reference, article["baseline_summary"])
            for metric, score in baseline_scores.items():
                metrics["baseline"][metric].append(score.fmeasure)
            
            # Score fine-tuned summary
            finetuned_scores = scorer.score(reference, article["finetuned_summary"])
            for metric, score in finetuned_scores.items():
                metrics["finetuned"][metric].append(score.fmeasure)
            
            # Compare ROUGE-L scores
            if finetuned_scores["rougeL"].fmeasure > baseline_scores["rougeL"].fmeasure:
                metrics["comparison"]["better"] += 1
            elif finetuned_scores["rougeL"].fmeasure < baseline_scores["rougeL"].fmeasure:
                metrics["comparison"]["worse"] += 1
            else:
                metrics["comparison"]["same"] += 1
    
    # Calculate averages
    for model in ["baseline", "finetuned"]:
        for metric in ["rouge1", "rouge2", "rougeL"]:
            if metrics[model][metric]:
                metrics[model][f"avg_{metric}"] = np.mean(metrics[model][metric])
            else:
                metrics[model][f"avg_{metric}"] = 0.0
    
    return metrics

# Evaluate summaries
if 'articles_with_summaries' in locals() and articles_with_summaries:
    evaluation = evaluate_summaries(articles_with_summaries)
    
    print("Evaluation Results:")
    print(f"ROUGE-1 Average (Baseline): {evaluation['baseline'].get('avg_rouge1', 0):.4f}")
    print(f"ROUGE-2 Average (Baseline): {evaluation['baseline'].get('avg_rouge2', 0):.4f}")
    print(f"ROUGE-L Average (Baseline): {evaluation['baseline'].get('avg_rougeL', 0):.4f}")
    print()
    print(f"ROUGE-1 Average (Fine-tuned): {evaluation['finetuned'].get('avg_rouge1', 0):.4f}")
    print(f"ROUGE-2 Average (Fine-tuned): {evaluation['finetuned'].get('avg_rouge2', 0):.4f}")
    print(f"ROUGE-L Average (Fine-tuned): {evaluation['finetuned'].get('avg_rougeL', 0):.4f}")
    print()
    print(f"Comparison: {evaluation['comparison']['better']} better, {evaluation['comparison']['worse']} worse, {evaluation['comparison']['same']} same")


## 7. Visualize Results

Let's create a visualization to compare the performance of our baseline and fine-tuned models.

In [ ]:
if 'evaluation' in locals():
    # Create DataFrame for plotting
    metrics = ["avg_rouge1", "avg_rouge2", "avg_rougeL"]
    baseline_values = [evaluation["baseline"].get(m, 0) for m in metrics]
    finetuned_values = [evaluation["finetuned"].get(m, 0) for m in metrics]
    
    df = pd.DataFrame({
        "Metric": ["ROUGE-1", "ROUGE-2", "ROUGE-L"],
        "Baseline": baseline_values,
        "Fine-tuned": finetuned_values
    })
    
    # Create plot
    plt.figure(figsize=(10, 6))
    df_melted = pd.melt(df, id_vars=["Metric"], var_name="Model", value_name="Score")
    
    ax = sns.barplot(x="Metric", y="Score", hue="Model", data=df_melted)
    plt.title("ROUGE Metrics Comparison")
    plt.ylim(0, 1.0)
    
    # Add value labels on bars
    for container in ax.containers:
        ax.bar_label(container, fmt='%.3f')
    
    plt.show()


## 8. Save Results

Finally, let's save our results to a JSON file for future reference or to display in the dashboard.

In [ ]:
if 'articles_with_summaries' in locals() and 'evaluation' in locals():
    # Prepare results
    results = {
        "timestamp": datetime.now().isoformat(),
        "portfolio": portfolio,
        "articles_count": len(articles_with_summaries),
        "articles": articles_with_summaries,
        "evaluation": evaluation
    }
    
    # Save results to file
    output_file = "../data/summaries.json"
    with open(output_file, 'w') as f:
        # Convert non-serializable objects
        # The json.dumps won't directly work with numpy values, so we need to convert them
        json_str = json.dumps(results, indent=2, default=lambda x: float(x) if isinstance(x, np.float32) else str(x))
        f.write(json_str)
    
    print(f"Results saved to {output_file}")


## 9. Conclusion

In this notebook, we've walked through the full process of the Financial News Summarization Agent:

1. We connected to the vector database to retrieve relevant articles for our portfolio
2. We set up baseline and fine-tuned summarization models
3. We generated and compared summaries for each article
4. We evaluated the quality of summaries using ROUGE metrics
5. We visualized the results to compare model performance

This process can be automated to run regularly, keeping you updated on the latest financial news related to your portfolio with high-quality summaries.

## 10. Next Steps

To extend this project further, you could:

1. **Automate news scraping**: Integrate with your Yahoo Finance scraper to continuously update the vector database
2. **Improve evaluation**: Add factual consistency checks and hallucination detection
3. **Fine-tune models**: Fine-tune a summarization model on your financial news dataset
4. **Create alerts**: Set up notifications for significant news about your portfolio
5. **Build a dashboard**: Use the news_dashboard.py script to visualize results in a web interface